In [8]:
%run "./ANN_Observer.ipynb"

✔️ The Setuper class succesfuly imported.
✔️ The ANN_Observer class succesfuly imported.


In [44]:
def capture_trace(input_data, cmd = 'p', print_ack = False):
        #Initial setup before capturing
        scope.arm() # arm the scope
        target.flush() # flush the UART buffer, normaly it is automatized, but in this case it is called manualy, it in essence flushes buffer storing incoming data t odevice
            
        #Send the data and capture
        target.send_cmd('p', 0x80, input_data)
        ret = scope.capture()
        wave = scope.get_last_trace()
        out_data = target.read_cmd('r') 
        out_data = None
        ack = target.read_cmd('e')
        trace = cw.Trace(wave, input_data, out_data, []) #save the metadata
        if print_ack:
            print(ack)
            print(out_data)  
        out_data = None
        
        return trace

def measure_traces(bnum = 8, input_size = 8, rand_indx = 0, rand_input = None, change_rand = False): #create_X_traces_with_rand_Nth_bit 
        #print(f"✔️ Starting measurement for {rand_indx}-th weights..")
        cw_setuper.reset_target(scope) # flush chipwhisperer and re-assing its parameters (this is moot, if used >1x)
        rand_inputs = rand_input
        traces = []
        if rand_inputs is None:
            print(f"⚠️ Generating rand_input..")
            L = range((2**bnum)-1) # Range of generated rand_numbers
            rand_inputs = [random.choice(L) for _ in range(10)]
        input_array = [np.uint8(0) for i in range(input_size)]
        
        # capture X (epochs) traces, while each has different rand_input
        for i in trange(epochs, desc='                                Capturing traces'):
            input_array[rand_indx] = np.uint8(rand_inputs[i])
            input_data = bytearray(input_array)
            trace = capture_trace(input_data=input_data)
            traces.append(trace)
        return traces

def scope_setup(samples=24431, decimate=2):
        # arm the scope
        scope.arm()
        # Set the maximum number of points in a trace
        scope.adc.fifo_fill_mode = "normal"
        scope.adc.samples = samples
        scope.adc.decimate = decimate

In [72]:
epochs = 10
bnum = 8
make_dir = "makefile_dir_basic_mult"
L = range((2**bnum)-1) # Range of generated rand_numbers
rand_inputs = [random.choice(L) for _ in range(epochs)]
re_init_config = {"MAKEDIR" : f'./{make_dir}/'}

In [86]:
cw_setuper = CW_Setuper(firmware_reinit = True, re_init_dict  = re_init_config)

⚠️ Changed, firmware config!!!
⚠️ Reinit, initiated!!!
✔️ Firmware maked.
INFO: Found ChipWhisperer😍
✔️ INFO: Scope and Target, are set😍
❤️ Target hex: ./makefile_dir_basic_mult/simpleserial-target-CWLITEARM.hex
Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more
Attempting to program 5087 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 5087 bytes
✔️ Setuper initiated.


In [87]:
scope = cw_setuper.scope
target = cw_setuper.target

In [88]:
scope_setup(samples=24430, decimate=1)    
traces = measure_traces()

⚠️ Generating rand_input..


                                Capturing traces:   0%|          | 0/10 [00:00<?, ?it/s]

In [89]:
waves = []
for i in traces:
    waves.append(i[0])

In [90]:
thousand_mult = waves[0]

In [91]:
cw.plot(tenthousand_mult) * cw.plot(thousand_mult)

:Overlay
   .Curve.I  :Curve   [x]   (y)
   .Curve.II :Curve   [x]   (y)

In [92]:
cw.plot(tenthousand_mult - thousand_mult)

:Curve   [x]   (y)

In [82]:
cw.plot(tenthousand_mult) * cw.plot(none_mult)

:Overlay
   .Curve.I  :Curve   [x]   (y)
   .Curve.II :Curve   [x]   (y)

In [83]:
cw.plot(tenthousand_mult - none_mult)

:Curve   [x]   (y)

In [81]:
none_mult = waves[0]

In [71]:
tenthousand_mult = waves[0]

In [85]:
cw_setuper._scope_disc_()

✔️ Firmware disconnected.
